In [1]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import autograd
from torch.utils import data
from torch.optim import Adam
from torchvision import datasets, transforms
import torchvision.models as models

In [2]:
# 난수 생성기가 항상 일정한 값을 출력하게 하기 위해 seed 고정
random_seed = 2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)

## EDA

In [3]:
DATASET_PATH = os.path.join('/USER/week2/data')

In [4]:
pd.read_csv(os.path.join(DATASET_PATH, 'train_edit.csv'))

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247.0,19128.0,2611.0,5161.0,1588.0,892.0,32263.0,1636.0,...,1311.0,3482.0,11299.0,7072.0,1176.0,3810.0,748.000000,3920.0,2133.0,3799.0
1,20200101,1,89309.0,19027.0,3337.0,5502.0,1650.0,1043.0,35609.0,1644.0,...,1162.0,3849.0,13180.0,8771.0,1283.0,3763.0,642.333333,3483.0,2057.0,4010.0
2,20200101,2,66611.0,14710.0,2970.0,4631.0,1044.0,921.0,26821.0,1104.0,...,768.0,2299.0,7986.0,5426.0,1536.0,3229.0,536.666667,2634.0,1526.0,3388.0
3,20200101,3,53290.0,13753.0,2270.0,4242.0,1021.0,790.0,21322.0,909.0,...,632.0,1716.0,5703.0,3156.0,1104.0,2882.0,431.000000,2488.0,1268.0,3686.0
4,20200101,4,52095.0,17615.0,2406.0,3689.0,1840.0,1080.5,22711.0,1354.0,...,875.0,2421.0,5816.0,2933.0,1206.0,2433.0,499.000000,2952.0,1927.0,5608.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,20200517,19,311727.0,101285.0,10085.0,30637.0,10060.0,8749.0,148935.0,6801.0,...,6726.0,15431.0,25597.0,14292.0,9300.0,22238.0,3786.000000,16936.0,10729.0,20194.0
3308,20200517,20,305354.0,91426.0,8607.0,26021.0,8095.0,7198.0,136503.0,6147.0,...,5501.0,15378.0,24661.0,14747.0,8239.0,20604.0,3203.000000,15018.0,9767.0,17962.0
3309,20200517,21,306008.0,75113.0,6325.0,19933.0,5711.0,4494.0,129412.0,5134.0,...,4216.0,12558.0,22781.0,14081.0,6392.0,17937.0,2447.000000,12403.0,7825.0,14031.0
3310,20200517,22,237447.0,49498.0,4209.0,12145.0,3891.0,2718.0,96698.0,3526.0,...,2578.0,8870.0,16640.0,11066.0,4427.0,11955.0,1495.000000,7507.0,5387.0,8889.0


In [7]:
df_val=pd.read_csv(os.path.join(DATASET_PATH, 'validate.csv'))

In [9]:
df_val

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200511,0,77968,14429,1233,4021,981,881,28672,1064,...,637,2604,5239,4168,1155,3596,337,2262,1608,2337
1,20200511,1,48679,9136,823,2618,654,572,17722,672,...,353,1870,3359,2558,1002,2157,257,1425,1018,1810
2,20200511,2,33773,8199,578,2188,392,502,14464,579,...,345,1499,2646,2022,876,1959,232,1155,927,1530
3,20200511,3,41511,9986,726,2817,555,646,17793,650,...,390,1730,3398,1967,912,2462,281,1477,959,1882
4,20200511,4,78680,19509,1463,4720,825,1088,35125,997,...,679,2958,7369,4120,1569,4568,577,3155,1871,3656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,...,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
332,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,...,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
333,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,...,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
334,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,...,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [6]:
pd.read_csv(os.path.join(DATASET_PATH, 'test.csv'))

,날짜,시간,10,100,101,120,121,140,150,160,...,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200518,0,82065,15172,1500,3294,1086,962,28931,1103,...,618,2790,5147,4331,1329,3665,404,2242,1619,2314
1,20200518,1,51248,9840,813,2356,696,546,17888,720,...,430,1864,3269,2561,921,2081,272,1390,1003,1766
2,20200518,2,39026,7894,760,2413,408,549,13357,498,...,322,1313,2765,1931,920,1764,228,1136,922,1309
3,20200518,3,40993,10137,780,2701,420,741,15544,532,...,326,1766,3320,2060,892,2447,337,1495,975,1912
4,20200518,4,77863,19603,1276,5019,968,1160,32101,968,...,669,2914,6986,3911,1368,4380,513,2940,1758,3629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200531,19,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
332,20200531,20,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
333,20200531,21,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
334,20200531,22,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


## Dataloader
* 한 칼럼에 대한 7일(168행) 데이터를 input_data, 뒤따르는 7일 데이터를 output_data로 반환합니다.
* 도로별 차이를 두지 않고 모든 도로를 동일한 타입의 데이터로 취급합니다.
* 모든 csv 파일의 마지막 168행은 예측해야하는 값이므로 input으로 들어가지 않습니다.

In [4]:
cluster_0 = ['150','500','1000']
cluster_1 =['100', '251', '352', '450', '550']
cluster_2 = ['101', '120', '121', '140', '160', '200', '201', '270', '300', '301',
       '351', '370', '400', '600', '650', '652', '1020', '1040', '1100',
       '1200', '1510', '2510', '3000', '4510', '5510', '6000']
cluster_3 = ['10']

## 클러스터 정보를 추가하기 위해 베이스라인 코드 수정

In [14]:
class CustomDataset(data.Dataset):      # torch.utils.data.Dataset 클래스의 상속 클래스 CustomDataset class 생성. 상속 클래스 생성시 __init__, __getitem__, __len__함수는 기본적으로 정의해줘야 함.
    
    def __init__(self, root, seq_len, cat, batch_size=64, phase='train'):      # 데이터 로드 단계에 사용될 여러 변수들을 'self.변수명'의 형태로 지정해두는 함수
        
        self.root = root      # CustomDataset 객체 생성 시 데이터 경로 앞부분(공통 부분)을 root로 입력받아 저장
        self.phase = phase      # CustomDataset 객체 생성 시 데이터 경로 뒷부분(train/validate/test)을 phase로 입력받아 저장
        self.label_path = os.path.join(self.root, self.phase + '.csv')      # 데이터 전체 경로 생성
        df = pd.read_csv(self.label_path)      # 생성한 데이터 전체 경로로부터 데이터 로드
        
        self.seq_len = seq_len * 24      # 일 단위 기간을 입력 받은 후 시간 단위 기간으로 변환하여 저장
        self.batch_size = batch_size
        self.labels = {}
        self.cat = cat                   
        self.highways = len(self.cat)
        timestamps = [(i, j) for (i, j) in zip(list(df['날짜']), list(df['시간']))]      # 날짜와 시간 정보가 튜플로 들어 있는 리스트 생성
        # categories = df.columns.values.tolist()[2:]      # 도로명 column list 생성

        input_data = []
        output_data = []

        for t in range(len(timestamps)):  
            temp_input_data = []
            temp_output_data = []
            for col in self.cat:
                road = df[col].tolist()        
                inp = [float(i) for i in road[t:t+self.seq_len]]      # input 데이터 시계열 구간 설정
                outp = [float(j) for j in road[t+self.seq_len:t+2*self.seq_len]]      # output 데이터 시계열 구간 설정
                temp_input_data.append(inp) 
                temp_output_data.append(outp)
            input_data.append(temp_input_data)
            output_data.append(temp_output_data)
            
# input_data : [[첫번째 input 기간 동안의 첫번째 도로의 통행량 list, ..., 첫번째 input 기간 동안의 35번째 도로의 통행량 list], ...,
#               [마지막 input 기간 동안의 첫번째 도로의 통행량 list, ..., 마지막 input 기간 동안의 35번째 도로의 통행량 list]]
# output_data : [[첫번째 output 기간 동안의 첫번째 도로의 통행량 list, ..., 첫번째 output 기간 동안의 35번째 도로의 통행량 list], ...,
#                [마지막 output 기간 동안의 첫번째 도로의 통행량 list, ..., 마지막 output 기간 동안의 35번째 도로의 통행량 list]]
        
        self.labels['timestamp'] = timestamps
        self.labels['category'] = self.cat
        self.labels['input'] = input_data
        self.labels['output'] = output_data

    def __getitem__(self, index):      # index를 가지고 데이터를 하나씩 불러올 수 있게 하는 함수

#         데이터 내 index가 부여되는 형태

#                 | road_1    road_2    ...  road_35
#                -------------------------------------
#         time_1  | index_0   index_1   ...  index_34
#         time_2  | index_35  index_36  ...  index_69

        row = index // self.highways      # index를 35(도로수)로 나눈 몫  ex) 71//35 -> 2
        col = index % self.highways      # index를 35(도로수)로 나눈 나머지  ex) 71%35 -> 1

        timestamp = self.labels['timestamp'][row]      # (날짜, 시간) 튜플이 들어있는 list에서 row번째 시점에 해당하는 튜플
        category = self.labels['category'][col]      # 도로명 column list에서 col번째 도로에 해당하는 element
        
        input_data = torch.tensor(self.labels['input'][row][col])      # input_data list에서, row번째 시점의 col번째 도로 교통량 정보

        if self.phase != 'test':
            output_data = torch.tensor(self.labels['output'][row][col])
        else:
            output_data = []

        return timestamp, category, (input_data, output_data)
    
    def __len__(self):      # getitem 함수를 통해 데이터를 불러오려면,전체 index 길이를 알아야 한다.
        return (len(self.labels['timestamp'])-(self.seq_len * 2)+1) * self.highways      # 특정 시점이 아닌 특정 기간을 하나의 data 단위로 설정하면, 전체 샘플 수는 감소함을 반영 


def data_loader(root,cat, phase='train', batch_size=64, seq_len=7, drop_last=False):
    if phase == 'train':
        shuffle = True
    else:
        shuffle = False
    cat =  cat
    dataset = CustomDataset(root, seq_len, cat, batch_size, phase)
    dataloader = data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader

## Model

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class LSTMNet(nn.Module):
    def __init__(self,
                 input_size=168,      # input 길이는 168시간(7일 X 24시간)
                 hidden_size=512,
                 output_size=168,      # output 길이는 168시간(7일 X 24시간)
                 batch_size=128,
                 num_layers=3,
                 dropout=0,
                 batch_first=False):      # batch_first(default=False) : 배치 차원을 첫번째 차원으로 하여 데이터를 불러올 것인지 여부

        super(LSTMNet, self).__init__()

        self.hidden_size = hidden_size
        
        ##### Layer 1
        self.lstm1 = nn.LSTM(input_size,
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        #### Layer 2
        self.lstm2 = nn.LSTM(hidden_size, 
                             hidden_size,
                             dropout=0.2,
                             num_layers=num_layers)

        ##### Finalize
        self.linear = nn.Linear(hidden_size, 
                                output_size)
        
        self.activation = nn.LeakyReLU(0.2)

        
    def forward(self, x, h_in, c_in):

        h_in = nn.Parameter(h_in.type(dtype), requires_grad=True)      # gradient descent로 업데이트 되는(requires_grad=True), h_in 이라는 이름의 파라미터 생성 
        c_in = nn.Parameter(c_in.type(dtype), requires_grad=True)      # gradient descent로 업데이트 되는(requires_grad=True), c_in 이라는 이름의 파라미터 생성

        # Layer 1
        lstm_out, (h_1, c_1) = self.lstm1(x, (h_in, c_in))
        lstm_out = self.activation(lstm_out)

        #Layer2
        lstm_out, (h_2, c_2) = self.lstm2(lstm_out, (h_1, c_1))
        lstm_out = self.activation(lstm_out)

        # Final
        predictions = self.linear(lstm_out)
        
        return predictions, (h_2, c_2)

In [7]:
# 결과 파일과 모델 가중치 파일 저장을 위해 log 디렉토리 생성. 중요한 파일이 덮어씌워지지 않도록 주의
os.makedirs('log', exist_ok=True)


def save_model(model_name, model, optimizer):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, os.path.join('log', model_name + '.pth'))
    print('model saved\n')
    return os.path.join('log', model_name + '.pth')


def load_model(model_name, model, optimizer=None):
    state = torch.load(os.path.join(model_name))
    model.load_state_dict(state['model'])
    if optimizer is not None:
        optimizer.load_state_dict(state['optimizer'])
    print('model loaded')

## Hyperparameters

In [25]:
dtype = torch.float
model_name = 'sequential'

batch_size = 128
num_epochs = 100
val_epoch = 1
base_lr = 0.01
seq_len = 7

input_size = seq_len * 24
output_size = input_size
hidden_size = 512
num_layers = 6

## Training Setting

In [30]:
# model
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)
model = model.to(device)

# loss function
criterion = nn.MSELoss()

# optimizer
optimizer = Adam(model.parameters(), lr=base_lr)      # optimizer로는 Adam이 가장 무난합니다. Adam을 쓰면 learning_rate를 따로 지정해주지 않아도 알아서 조정됩니다.

In [31]:
print(model)

LSTMNet(
  (lstm1): LSTM(168, 512, num_layers=6, dropout=0.2)
  (lstm2): LSTM(512, 512, num_layers=6, dropout=0.2)
  (linear): Linear(in_features=512, out_features=168, bias=True)
  (activation): LeakyReLU(negative_slope=0.2)
)


In [32]:
# get data loader
train_dataloader = data_loader(root=DATASET_PATH,
                               cat= cluster_3,
                               phase='train_edit',
                               batch_size=batch_size,
                               seq_len=seq_len,
                               drop_last=True)

validate_dataloader = data_loader(root=DATASET_PATH,
                                  cat= cluster_3,
                                  phase='validate',
                                  batch_size=1,
                                  seq_len=seq_len,
                                  drop_last=True)

In [48]:
a = CustomDataset( DATASET_PATH, seq_len,cat=cluster_3, batch_size=64, phase='train')
b = CustomDataset( DATASET_PATH, seq_len,cat=cluster_3, batch_size=64, phase='validate')


In [87]:
a[0]

((20200101, 0),
 '10',
 (tensor([ 83247.,  89309.,  66611.,  53290.,  52095.,  62591.,  72799.,  98367.,
          138272., 204720., 285893., 324836., 320617., 331264., 359475., 384305.,
          387721., 354546., 299045., 272186., 256661., 230300., 170933., 107623.,
           62778.,  40492.,  32815.,  36717.,  64090., 142571., 248187., 321245.,
          302631., 282018., 307976., 324038., 321212., 331174., 337396., 340639.,
          338451., 333273., 298945., 241560., 217665., 194461., 157652., 112308.,
           82799.,  59306.,  48045.,  49712.,  67423., 115647., 194381., 280641.,
          299510., 300402., 320228., 332667., 333576., 340362., 366144., 386427.,
          400061., 421336., 407648., 345687., 306051., 268332., 208612., 144902.,
           97966.,  69127.,  52807.,  47297.,  56987.,  90089., 144122., 199788.,
          268007., 338748., 385538., 387355., 356511., 363416., 388014., 399648.,
          401181., 378500., 321310., 259903., 232601., 210189., 163773., 11

((20200518, 3),
 '352',
 (tensor([ 6202., 11987., 34498., 58310., 74453., 68887., 65707., 66946., 66569.,
          63158., 60856., 66447., 66419., 61933., 58214., 59437., 41850., 41873.,
          29095., 19299., 13873., 10253.,  6710.,  6184.,  6578.,  9001., 20300.,
          36178., 52825., 59803., 61520., 59795., 61635., 57030., 59370., 61211.,
          65022., 66107., 63431., 56580., 41155., 32446., 27445., 20854., 15498.,
          10810.,  7677.,  6931.,  7124., 11882., 28467., 48141., 61618., 64950.,
          69321., 72998., 70947., 68448., 68544., 69960., 71414., 74861., 73614.,
          69255., 50034., 39784., 31584., 23496., 16309., 10679.,  7707.,  6968.,
           7049., 11152., 27273., 44909., 60945., 67650., 71229., 74439., 72799.,
          66742., 62039., 67227., 68661., 66022., 67204., 64519., 51026., 40895.,
          33007., 22449., 15423., 10555.,  7312.,  6510.,  6217., 10605., 24691.,
          41818., 55367., 60688., 64822., 71262., 72857., 70202., 74147., 

In [92]:
for iter_, sample in enumerate(train_dataloader):
    _, _, (input_data, output_data) = sample
    print(len(input_data))
    print(input_data.shape)
    print(input_data.unsqueeze(0).shape)
    break

64
torch.Size([64, 168])
torch.Size([1, 64, 168])


## Train

일단 혼자 분류된 10번도로만 학습을 잘하는지 돌려보았으나 오차가 너무커서 LSTM은 성능이 안나온다고 판단함

In [33]:
train_batch_loss = 0.0
train_epoch_loss = 0.0

valid_epoch_loss = 0.0
valid_min_epoch_loss = np.inf

for epoch in range(num_epochs):

    model.train()      # 모델을 train mode로 전환. train mode일 때만 적용되어야 하는 drop out 등이 적용될 수 있게 하기 위함 

    for iter_, sample in enumerate(train_dataloader):      # enumerate 함수를 통해 train_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

        (h_in, c_in) = (torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device),
                        torch.zeros(num_layers, batch_size, hidden_size, requires_grad=True).to(device))

        _, _, (input_data, output_data) = sample      # train_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨. 학습에는 [[input_data], [output_data]]만 사용
        
        input_data = input_data.unsqueeze(0).to(device)
        output_data = output_data.unsqueeze(0).to(device)

        pred, (h_in, c_in) = model(input_data, h_in, c_in)
        
        loss = criterion(pred, output_data)

        model.zero_grad()    # 파라미터 업데이트는 batch 단위로 이루어지고, 매 batch마다 gradient를 초기화해주어야 함 
        loss.backward()      # backpropagation
        optimizer.step()      # 파라미터 업데이트
        
        train_batch_loss += loss.item()
        train_epoch_loss += loss.item()

        if iter_ % 400 == 399:      # 400개의 batch마다 training Loss 출력
            print('Train Epoch: {:2} | Batch: {:4} | Loss: {:1.2f}'.format(epoch, iter_+1, train_batch_loss/400))
            train_batch_loss = 0
            
    train_epoch_loss = 0.0

    
    model.eval()      # 모델을 eval mode로 전환. eval mode에서 적용되면 안되는 drop out 등이 적용되지 않게 하기 위함

    with torch.no_grad():      # validation / test set에 대해서는 weight 및 bias의 update, 즉, gradient descent가 일어나지 않도록 no_grad()를 선언
        (h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                        torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

        for iter_, sample in enumerate(validate_dataloader):      # enumerate 함수를 통해 validate_dataloader에서 'batch의 index'와 'batch'를 순서대로 호출

            _, _, (input_data, output_data) = sample      # validate_dataloader에서 불러온 sample은 [[날짜, 시간], [도로], [[input_data],[output_data]]]로 구성됨. validation에는 [[input_data], [output_data]]만 사용

            input_data = input_data.unsqueeze(0).to(device)
            output_data = output_data.unsqueeze(0).to(device)

            pred, (h_in, c_in) = model(input_data, h_in, c_in)
            loss = criterion(pred, output_data)
            valid_epoch_loss += loss.item()

        print('\nValid Epoch: {:2} | Loss: {:1.2f}'.format(epoch, valid_epoch_loss/len(validate_dataloader)))

        if valid_epoch_loss < valid_min_epoch_loss:
            save_model('best', model, optimizer)
            valid_min_epoch_loss = valid_epoch_loss

        valid_epoch_loss = 0.0


Valid Epoch:  0 | Loss: 71074865152.00
model saved


Valid Epoch:  1 | Loss: 71056515072.00
model saved


Valid Epoch:  2 | Loss: 71038230528.00
model saved


Valid Epoch:  3 | Loss: 71020142592.00
model saved


Valid Epoch:  4 | Loss: 71002169344.00
model saved


Valid Epoch:  5 | Loss: 70984278016.00
model saved


Valid Epoch:  6 | Loss: 70966435840.00
model saved


Valid Epoch:  7 | Loss: 70948634624.00
model saved


Valid Epoch:  8 | Loss: 70930866176.00
model saved


Valid Epoch:  9 | Loss: 70913122304.00
model saved


Valid Epoch: 10 | Loss: 70895411200.00
model saved


Valid Epoch: 11 | Loss: 70877700096.00
model saved


Valid Epoch: 12 | Loss: 70860021760.00
model saved


Valid Epoch: 13 | Loss: 70842343424.00
model saved


Valid Epoch: 14 | Loss: 70824681472.00
model saved


Valid Epoch: 15 | Loss: 70807027712.00
model saved


Valid Epoch: 16 | Loss: 70789390336.00
model saved


Valid Epoch: 17 | Loss: 70771761152.00
model saved


Valid Epoch: 18 | Loss: 70754140160.00
model 

## Inference

In [16]:
dtype = torch.float
seq_len = 7

input_size = seq_len * 24
hidden_size = 1024
output_size = input_size
batch_size = 1
num_layers = 6

In [17]:
test_dataloader = data_loader(root=DATASET_PATH,
                              phase='test',
                              batch_size=batch_size,
                              seq_len=seq_len,
                              drop_last=True)

In [18]:
model = LSTMNet(input_size=input_size,
                hidden_size=hidden_size,
                output_size=output_size,
                batch_size=batch_size,
                num_layers=num_layers)

# model
model_name = 'log/best.pth'

load_model(model_name, model)
model = model.to(device)

model loaded


In [19]:
submission_file_path = os.path.join(DATASET_PATH, 'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)

In [20]:
(h_in, c_in) = (torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device),
                torch.zeros(num_layers, 1, hidden_size, requires_grad=False).to(device))

for iter_, sample in enumerate(test_dataloader):

    timestamp, category, (input_data, output_data) = sample
    input_data = input_data.unsqueeze(0).to(device)

    pred, (h_in, c_in) = model(input_data, h_in, c_in)

    for i, (t, h) in enumerate(zip(timestamp[0], timestamp[1])):
        for cat, row in zip(category, pred[0]):
            cat = f'{cat}'
            submission_table[cat] = row.tolist()

In [21]:
submission_table.to_csv('prediction_drop.csv', index=False)

tensor([[[ 2337.,  1810.,  1530.,  1882.,  3656.,  9440., 19625., 27199.,
          24872., 21953., 22573., 22182., 19528., 19447., 20903., 21353.,
          22821., 24493., 22935., 15875., 10581.,  7361.,  5305.,  3745.,
           2476.,  1667.,  1703.,  1744.,  3588.,  7436., 15250., 24448.,
          23720., 22650., 23055., 22253., 19605., 19215., 21583., 22555.,
          23532., 25202., 23263., 14596., 10501.,  7269.,  5204.,  3758.,
           2233.,  1691.,  1384.,  1928.,  3415.,  8062., 15565., 24127.,
          24285., 22211., 23793., 22991., 19905., 19955., 21720., 22275.,
          23603., 24844., 16339., 15929., 10519.,  7940.,  5640.,  3769.,
           2622.,  1511.,  1644.,  1789.,  2946.,  7822., 15242., 23926.,
          23276., 22049., 23750., 21962., 19574., 20372., 21838., 22048.,
          24353., 26552., 25576., 16449., 12398.,  8502.,  6246.,  4312.,
           2368.,  1696.,  1590.,  1866.,  2673.,     0., 12483., 20530.,
          21995., 19800., 21407., 2091